In [1]:
import cv2  
import threading  
import time  
from ultralytics import YOLO  
from queue import Queue 



c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
model = YOLO("yolov8n-pose.pt") 
cap = cv2.VideoCapture("popit-vodichki.mp4")
start_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        results = model(frame, imgsz=[640, 480], save=False, verbose=False)
        annotated_frames = results[0].plot()
        # cv2.imshow("Inference", cv2.resize(annotated_frames, (640, 480)))
        # cv2.imshow("Inference", annotated_frames)
           
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
end_time = time.time()
time_single = end_time - start_time
cap.release()
cv2.destroyAllWindows()

In [5]:

model = YOLO("yolov8n-pose.pt") 
cap = cv2.VideoCapture("popit-vodichki.mp4")
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(width, height)

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
out = cv2.VideoWriter("output_video_single.mp4", fourcc, 30.0, (width, height))
start_time = time.time()
while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        results = results = model(frame, save=False, verbose=False) 
        annotated_frames = results[0].plot()
        out.write(annotated_frames)
        # if cv2.waitKey(1) & 0xFF == ord('q'):
        #     break
    else:
        break
end_time = time.time()
time_single = end_time - start_time
cap.release()
cv2.destroyAllWindows()


576 1024


# Single_no_resize_imshow

In [4]:
time_single

42.29179930686951

# Single_resize_imshow

In [6]:
time_single

37.35465431213379

# Single_save_no_imshow

In [9]:
model = YOLO("yolov8n-pose.pt") 
video_path = "popit-vodichki.mp4"
start_time = time.time()
results = model(video_path, save=True, verbose=False)

end_time = time.time()
time_single = end_time - start_time


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

Results saved to runs\pose\predict2


In [10]:
video_path = "popit-vodichki.mp4"
cap = cv2.VideoCapture(video_path)
frames = []
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame)
cap.release()

In [15]:
frames[0].shape

(1024, 576, 3)

In [11]:
len(frames)

381

In [21]:
num_threads = 2
chunk_size = len(frames)
threads = []

start_time = time.time()

for i in range(num_threads):
    chunk = frames[i * chunk_size: (i + 1) * chunk_size] if i < num_threads - 1 else frames[i * chunk_size:]
    print(chunk[0].shape)

(1024, 576, 3)


IndexError: list index out of range

# Threading_Queue_dict num_thread = 4

In [3]:
def process_video_multi_thread(video_path, num_threads):    
    cap = cv2.VideoCapture(video_path)    
    frames = []    
        
    while cap.isOpened():    
        ret, frame = cap.read()    
        if not ret:    
            break    
        frames.append(frame)    
    cap.release()    

    results_dict = {}  

    def worker(frame_queue):    
        model = YOLO("yolov8n-pose.pt")   
        while True: 
            index, frame = frame_queue.get() 
            if frame is None: 
                break 
            results = model(frame, save=False, verbose=False)   
            annotated_frame = results[0].plot()   
            results_dict[index] = annotated_frame  
            frame_queue.task_done() 

    frame_queue = Queue(maxsize=0) 
     
    threads = [] 
    for i in range(num_threads):    
        thread = threading.Thread(target=worker, args=(frame_queue,))    
        threads.append(thread)    
        thread.start()    

    start_time = time.time()    

    for index, frame in enumerate(frames): 
        frame_queue.put((index, frame))  

    frame_queue.join() 

    for _ in range(num_threads): 
        frame_queue.put((None, None)) 

    for thread in threads:    
        thread.join()

    height, width, _ = frames[0].shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
    out = cv2.VideoWriter("output_video.mp4", fourcc, 30.0, (width, height))

    delay = int(1000/30) 
    for index in range(len(frames)): 
        out.write(results_dict[index])
        cv2.imshow("Inference", results_dict[index]) 
        cv2.waitKey(delay)   

    cv2.destroyAllWindows()    
    end_time = time.time()    
    return end_time - start_time   

process_video_multi_thread("popit-vodichki.mp4", num_threads=4)


52.26506447792053

In [3]:
process_video_multi_thread("popit-vodichki.mp4", num_threads=2)

64.263512134552

In [4]:
process_video_multi_thread("popit-vodichki.mp4", num_threads=6)

48.03016233444214

In [5]:
process_video_multi_thread("popit-vodichki.mp4", num_threads=12)

47.78323197364807

In [6]:
process_video_multi_thread("popit-vodichki.mp4", num_threads=30)

Exception in thread Exception in thread Thread-79 (worker):
Traceback (most recent call last):
  File "c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-76 (worker):
Traceback (most recent call last):
  File "c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-55 (worker):
Traceback (most recent call last):
  File "c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-45 (worker):
Traceback (most recent call last):
  File "c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-71 (worker):
Traceback (most recent call last):
  File "c:\Users\Po4ka\AppData\Local\Programs\Python\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
Exception in thread Thread-97 (worker):


: 